In [ ]:
import numpy as np
from scipy.sparse import lil_matrix, dok_matrix, csr_matrix
from collections import defaultdict, OrderedDict
from random import randint, choice
from pprint import pprint as pp
from array import array
from itertools import product, combinations
from time import perf_counter
from copy import copy
from time import sleep

# Setup (Metrics)

In [ ]:
import psutil
import os

def mem():
    """ Use psutil to record memory snapshot. """
    pid = os.getpid()
    p = psutil.Process(pid)
    rss, vms = p.memory_info()
    return vms

class Stats:
    """ Context manager for reporting memory change and time cost. """
    def __enter__(self):
        self.m1 = mem()
        self.t1 = perf_counter()
        
    def __exit__(self, type, value, traceback):
        self.t2 = perf_counter()
        self.m2 = mem()
        print('\nChange in memory: ', end='')
        print('{:.4g} MB'.format((self.m2 - self.m1) / 1024 / 1024))
        print('Time cost (s)   : ', end='')
        print('{:.4g} s\n'.format(self.t2 - self.t1))
    
# Demo!
with Stats():
    x = [0]*100000000  # 100M
    
with Stats():
    del x

# Create the list of context blocks

In [ ]:
words = [_.strip() for _ in open('/usr/share/dict/words', 'r')]
print('Number of unique words: %d\n' % len(words))
words = words[:61000]  # Truncate the list to be more realistic

### Create our own hash for bidirectional lookups

In [ ]:
# Give word, get index
# This is the opposite of `words`: give index, get word
wordsd = OrderedDict(zip(words, range(len(words))))

In [ ]:
# Test
x = words[1234]
print(x)
print(wordsd[x])
words[32751]

## Dataset creation

In [ ]:
def make_context_blocks(num_blocks=100000, word_count=(5, 20)):
    context_blocks = []
    for i in range(num_blocks):
        block_size = choice(range(*word_count))
        #
        # Pretty important that `sorted` is called here. This makes 
        # combinations stable later.
        #
        block = sorted(set(choice(words) for i in range(block_size)))
        context_blocks.append(block)
    return context_blocks
             
with Stats():
    context_blocks = make_context_blocks()
    
print('Sample blocks:')
for b in context_blocks[:5]:
    print(' - ', '/'.join(b))

### Build a version of context_blocks that is only arrays of arrays

This changes the context blocks, i.e. the list of lists of 5-40 strings, into a two-dimensional array of integers. Each integer is an index into the `idx` hash that was built earlier.

The array **is preallocated** for both rows and columns.  Currently we're using a default of 100 for columns.  This easily covers the 5-40 band, obviously.  We use "-1" as default, and this is used to know which entries are valid words and which are not.

In [ ]:
def make_cb_array(context_blocks, max_words_per_block=100):
    # Note: values are initialized to -1.  This is to keep track of 
    # which entries are valid. These will be >=0, and will index into
    # the `words` list.
    context_blocks_array = np.zeros(
        (len(context_blocks), max_words_per_block), 
        dtype='i4') - 1
    for i, block in enumerate(context_blocks):
        for j, word in enumerate(block):
            # wordsd is a reverse lookup. You give the word, it tells
            # you the index in the "words" array.
            context_blocks_array[i, j] = wordsd[word]
    return context_blocks_array

context_blocks_array = make_cb_array(context_blocks)

In [ ]:
# Demo
print(context_blocks_array[500])
for i in context_blocks_array[500]:
    if i>-1:
        print(words[i], end=', ')

# Naive `dict` method.  Dicts inside Dicts

(Also, this is all working with strings.  See further down for using naive dicts but with integers everywhere.)

In [ ]:
def method_dict(context_blocks):
    """
    Given a list of blocks (each containing 5-40 words), build a dict that 
    itself contain dicts. the inner dict has a count of the number of associations
    between the outer key and the inner key.
    """
    d = defaultdict(lambda: defaultdict(int))
    for block in context_blocks:
        for w1, w2 in combinations(block, 2):
            d[w1][w2] += 1
    return d

with Stats():
    d = method_dict(context_blocks)
    
associations = sum(len(w2s) for w1, w2s in d.items())
print('associations: ', associations)

Using `setdefault` all over the place is slower, but really not by much.

In [ ]:
def method_dict2(context_blocks):
    """
    Given a list of blocks (each containing 5-15 words), build a dict that 
    itself contain dicts. the inner dict has a count of the number of associations
    between the outer key and the inner key.
    """
    d = {}
    for block in context_blocks:
        for w1, w2 in combinations(block, 2):
            d.setdefault(w1, {})
            d[w1].setdefault(w2, 0)
            d[w1][w2] += 1
    return d

with Stats():
    d2 = method_dict(context_blocks)
    
associations = sum(len(w2s) for w1, w2s in d2.items())
print('associations: ', associations)

In [ ]:
# Show a sample of the resulting dict.
for i, (w1, w2s) in enumerate(d.items()):
    if i > 5:
        break
    print(w1)
    for j, w2 in enumerate(w2s):
        if j > 5:
            break
        print(' '*8, '{:20} {:10}'.format(w2, d[w1][w2]))

# Using a counter

In [ ]:
from collections import Counter

def method_counter(context_blocks):
    c = Counter()
    for block in context_blocks:
        c.update(combinations(block, 2))
    return c

with Stats():
    cnt = method_counter(context_blocks)
print('Associations:',len(cnt))
print()

In [ ]:
for iter, ((w1, w2), c) in enumerate(cnt.items()):
    print('{:15}{:15}{:4}'.format(w1, w2, c))
    if iter>5:
        break

# Cython (naive) - Also using dicts

In [ ]:
%load_ext cython

In [ ]:
%%cython -a

import numpy as np

def method_cython1(list context_blocks):
    """
    Given a list of blocks (each containing 5-40 words), build a dict that 
    itself contain dicts. the inner dict has a count of the number of associations
    between the outer key and the inner key.
    """
    #cdef int n = int(100e6)
    #cdef unsigned int[:] w1 = np.zeros(n, dtype='u4') - 1
    #cdef unsigned int[:] w2 = np.zeros(n, dtype='u4') - 1
    cdef int end = 0, i, j, blen
    cdef list block
    cdef dict out = {}, inner
    cdef str w1, w2
    for block in context_blocks:
        blen = len(block)
        for i in range(blen):
            w1 = block[i]
            inner = out.get(w1) or {}
            for j in range(i+1, blen):
                w2 = block[j]
                if not w2 in inner:
                    inner[w2] = 0
                inner[w2] += 1
            out[w1] = inner
    return out

In [ ]:
with Stats():
    d = method_cython1(context_blocks)
    
associations = sum(len(w2s) for w1, w2s in d.items())
print('associations: ', associations)

# Numpy

A quick demo of how to use the integer version of the context blocks.

In [ ]:
# Take on particular block
a = context_blocks_array[500]
# Take only the assigned words from the block (drop "-1"s)
b = a[a>-1]
print('Words in this block:\n\n',b, end='\n'*2)
x = np.zeros(200, dtype='i4')
x[5:5+len(b)] = b
print('Pair combinations of these words:', end='\n'*2)
for _ in list(combinations(b, 2)):
    print(_, end=",")

### Tools for the numpy work: faster combinations, and `lru_cache`

In [ ]:
from scipy.misc import comb
from itertools import chain
from functools import lru_cache

# The basic strategy is to build INDICES of 
# combinations, and then use Numpy's clever
# index assignment to generate the actual 
# combinations arrays.

@lru_cache()
def comb_index(n, k):
    count = comb(n, k, exact=True)
    index = np.fromiter(chain.from_iterable(combinations(range(n), k)), 
                        'i4', count=count*k)
    return index.reshape(-1, k)

def combb(data):
    idx = comb_index(len(data), 2)
    return data[idx]

# It turns out that 2-combinations are efficiently produced via an upper
# triangluar array. Other than that, same as before, we first calculate
# the INDICES array, and then pass that into our data to build the
# actual list of combinations.

@lru_cache()
def comb_index_triu(n, k):
    return np.array(np.triu_indices(n, 1)).T
    
def combtriu(data):
    idx = comb_index_triu(len(data), 2)
    return data[idx]

print('Compare the first few elements of each combinations function:', end='\n\n')
print(combb(b[:3]))
print(combtriu(b[:3]))

## Basic numpy method.  All arrays, uses fast combinations functions.

In [ ]:
import numpy as np
from scipy.misc import comb

def method_numpy1(context_blocks_array, max_words_per_block=40):
    """
    We create one, very long array (many rows) with 2 columns.  Every time
    we add a co-occurence, we simply use a new row to record the two words.
    There are some clever tricks inside the sub methods, mostly about how 
    to work with the combinations efficiently, but basically this pretty 
    much just records every co-occurence in a pretty dumb way.
    
    It turns out this is also quite fast.
    
    Note that we DON'T sum the counts here.  This means that the output 
    array will have duplicated pairs. IOW there will be multiple rows
    with the same two entries.  Afterwards, you will have to sum the
    duplicates to determine the co-occurence counts.
    """
    # Pre-allocation of array: WORST CASE
    p = comb(max_words_per_block, 2)
    n = int(len(context_blocks_array) * p)
    print('Worst-case pre-allocation is {:,} entries.'.format(n))
    co = np.zeros((n, 2), dtype='i4') - 1
    end = 0  # Keep track of position in the allocation array

    for block in context_blocks_array:
        # Combinations of words in this block. (m, 2) array
        new_entries = combtriu(block[block>-1])  
        # Copy the new associations directly in
        co[end:end+len(new_entries), :] = new_entries
        # Move the "current position" marker
        end += len(new_entries)
    
    # Return an array of the correct size (truncate)
    print('Actual count turned out to be {:,} entries.'.format(end+1))
    return co[:end, :]

### Performance Test

In [ ]:
try:
    del co
except:
    pass

with Stats():
    co = method_numpy1(context_blocks_array)
    
associations = len(co)
print('associations: {:,}'.format(associations))

### How to use the output?  Use slicing.

In [ ]:
# Demo of use
def top_cooccurences(co, word, most_common_count=10):
    """ 
        co: one big array (n x 2).  Each entry is an individual co-occurence.
        word: A word that you want to find the co-occurences for.
        most_common_count: The number of most common co-occurences to return.
        
    You give a word, this function returns the 
    other words most strongly associated with
    it, along with the counts.
    """
    ix = wordsd[word]
    # Find most common pair with "capivi"
    entries_above = co[co[:,0]==ix]
    entries_below = co[co[:,1]==ix]

    single_array = np.concatenate((entries_above[:,1], entries_below[:,0]), axis=0)  
    idx, counts = np.unique(single_array, return_counts=True)
    
    other_words = [words[idx[_]] for _ in range(most_common_count)]
    return other_words, counts[:most_common_count]

In [ ]:
top_cooccurences(co, 'capivi', 3)

To find the most common associations in the entire result, you would have to build a sparse array to count them.

**Note that the act of building the sparse array will also count duplicate entries automatically. It's doing some of our work for us basically.**

In [ ]:
def make_sparse(co):
    return csr_matrix(
            (np.ones(co.shape[0], dtype='u4'), (co[:,0], co[:,1])),
            dtype='u4')

m = make_sparse(co)
m.shape

Now we can query the top counts across the entire array quite easily.

In [ ]:
# All entries with a cooccurence > 2
# The two arrays returned are indexes for each dimension.
m[m>2].nonzero()

If you're **only interested in rows**, you could also sum the array across the columns and see what comes up.

In [ ]:
sums = m.sum(axis=1)
max_count_index = sums.argmax()
max_count_value = sums[max_count_index, 0]
print('Word with the biggest count is {} with {}.'.format(max_count_index, max_count_value))
print('(That word is {})'.format(words[max_count_index]))

What if you want to find the top 10 **ROWS**?

In [ ]:
def top_rows(m: "sparse array", count=10):
    sums = m.sum(axis=1).ravel()
    #print(sums.shape, sums)
    indices = np.argsort(sums, 1)
    #print(indices.shape, indices)
    indices = indices[0, -count:]
    #print(indices.shape, indices)
    #print(indices[0, -2])
    for ix in range(-1, -count-1, -1):
        i = indices[0, ix]
        print('{:20} : {:<}'.format(words[i], sums[0,i]))
        
# Demo
print('The top 2:')
print('==========')
top_rows(m, 2)
print()
print('The top 10:')
print('===========')
top_rows(m, 10)

# Very large test

In [ ]:
with Stats():
    new_cb = make_context_blocks(num_blocks=int(2e5), word_count=(5,40))
    
with Stats():
    new_cba = make_cb_array(new_cb)

print(len(new_cb), len(new_cba))

In [ ]:
try:
    del co2
except:
    pass

with Stats():
    co2 = method_numpy1(new_cba)
    
print('Associations  : ','{0:,}'.format(len(co2)))
print('Size of result: {:,.2f} MB'.format(co2.nbytes/1024/1024))
    

### Check out the top 5 rows

In [ ]:
with Stats():
    m = make_sparse(co2)
    
print('Size of sparse matrix: {:,.2f} MB'.format(m.data.nbytes/1024/1024))
print()
print('Top 5 rows (words):')
print()
    
with Stats():
    top_rows(m, 5)

# What about `dict` but with ints and our numpy tools?

The results are pretty bad, surprisingly so.  Needs more investigation to figure out why.

In [ ]:
def method_dict_int(context_blocks_array):
    """
    Given a list of blocks (each containing 5-40 words), build a dict that 
    itself contain dicts. the inner dict has a count of the number of associations
    between the outer key and the inner key.
    
    THIS ONE USES INTEGERS EVERYWHERE.
    """
    d = defaultdict(lambda: defaultdict(int))
    for block in context_blocks_array:
        for w1, w2 in combtriu(block[block>-1]):
            d[w1][w2] += 1
    return d

with Stats():
    d = method_dict_int(context_blocks_array)
    
associations = sum(sum(w2s.values()) for w1, w2s in d.items())
print('associations: ', associations)
print(len(context_blocks_array))

# Sparse

In [ ]:
import numpy as np
from scipy.misc import comb

def method_sparse(context_blocks_array, max_words_per_block=40, max_section_length=int(1e7)):
    """
    Series of sparse matrix constructions.
    
    max_section_length is a setting.  Tweak to trade-off CPU vs RAM.
    """    
    # Max combinations possible in each block
    p = comb(max_words_per_block, 2)     
    
    # Buffers 
    ones = np.ones(max_section_length, dtype='u2')
    co = np.zeros((max_section_length, 2), dtype='u2')
    end = 0  # Keep track of position in the allocation array 
    
    # The max number of unique words.  Might need to go up.
    # Sets num rows and cols for the output sparse matrix
    ns = 2**16  # (65536) 
    # Output. Stores co-occurrence totals between word pairs.
    # The datatype determines the max count possible, and also the 
    # memory cost of the sparse matrix.  'u2' is quite aggressively
    # small. u4 shouldn't be much worse.
    m = csr_matrix((ns, ns), dtype='u2')  # 
    
    for block in context_blocks_array:
        # Combinations of words in this block.
        new_entries = combtriu(block[block>-1])  
        #new_entries = combb(block[block>-1]) 
        # Copy the new associations directly in
        co[end:end+len(new_entries), :] = new_entries
        # Move the "current position" marker
        end += len(new_entries)
        # Buffer might be full
        full = end > max_section_length - p  # Account for next iteration fill-up, worst case
        if full:
            m += csr_matrix((ones[:end], (co[:end, 0], co[:end, 1])), (ns, ns))
            end = 0 # Reset back to start
    
    if end > 0:
        m += csr_matrix((ones[:end], (co[:end, 0], co[:end, 1])), (ns, ns))
    return m    
    
try:
    del m
except:
    pass

print('Length of context_blocks_array:',len(context_blocks_array))
with Stats():
    m = method_sparse(context_blocks_array[:100000], max_section_length=int(1e7))
    
print('Total co-occurences: {:,}'.format(m.sum()))
print('Size of sparse matrix: {:,.2f} MB'.format(m.data.nbytes/1024/1024))

### Using the sparse array

In [ ]:
with Stats():
    top_rows(m)

## Try out the big one

In [ ]:
try:
    del m
except:
    pass

print('Length of context_blocks_array:',len(new_cba))
with Stats():
    m = method_sparse(new_cba, max_section_length=int(1e6))
print('Total co-occurences: {:,}'.format(m.sum()))
print('Size of sparse matrix: {:,.2f} MB'.format(m.data.nbytes/1024/1024))

### Memory is great but it's a bit on the slow side.

We can increase the buffer size, reducing the number of times a sparse matrix has to be built internally.  Let's search for the optimum.

In [ ]:
try:
    del m
    #sleep(0)
except:
    pass

print('Length of context_blocks_array:',len(new_cba))
for i in range(1,11):
    size = int(i*1e7)
    print('*********************')
    print('Buffer size: {:,}'.format(size))
    print('*********************')
    with Stats():
        m = method_sparse(new_cba, max_section_length=size)
    print('Total co-occurences: {:,}'.format(m.sum()))
    print('Size of sparse matrix: {:,.2f} MB'.format(m.data.nbytes/1024/1024))

Looks like we get our best timings with a buffer length of 3e7.

# Sparse + Cython

The sparse option seems to work quite well.  Here we'll try to optimize it using Cython.  The main thing is to remove all interaction with the Python runtime inside the inner loops.

### First make some utilities

In [ ]:
%%cython -a
# cython: language_level = 3
cimport cython
cimport numpy as np
import numpy as np

@cython.cdivision(True)
cdef inline long comb_count_safe(long n, long k):
    cdef long i, prod = 1
    for i in range(k):
        # The bracketing is super-important. Order of operations matters.
        prod = (prod * (n - i)) / (i + 1)
        #print(n-i, i+1, prod)
    return prod
    
def comb_cy(int n):
    cdef int i, j
    for i in range(n):
        for j in range(i+1,n):
            print(i,j)
            
cdef inline object comb_cy2(unsigned short n):
    cdef int i, j, row = 0
    cdef unsigned short[:,:] out = np.zeros((comb_count_safe(n,2), 2), dtype='u2')
    for i in range(n):
        for j in range(i+1,n):
            #print(i,j)
            out[row,0] = i
            out[row,1] = j
            row += 1
    return np.array(out)
          
    
def make_lookup_array(unsigned short max_n) -> list:
    cdef int i, j, k, n, row = 0
    cdef int maxcol = comb_count_safe(max_n, 2)
    #cdef list out = [0]
    cdef unsigned short[:,:,:] out = np.zeros((max_n+1, maxcol + 1, 2), dtype='u2')
    cdef unsigned short[:,:] tmp
    for i in range(1, max_n + 1):
        n = comb_count_safe(i, 2)
        #print(comb_cy2(i))
        tmp = comb_cy2(i)
        for j in range(n):
            for k in range(2):
                out[i, j, k] = tmp[j, k]
        out[i, maxcol, 0] = n
        #out.append(comb_cy2(i))
    return np.array(out)
    
comb_cy(5)
print('safe',comb_count_safe(40,2))

from scipy.misc import comb
print('scipy', comb(40, 2))

print(comb_cy2(5))

In [ ]:
x = make_lookup_array(5)
print(x.shape, x[4, 10, 0])

for i, r in enumerate(make_lookup_array(5)):
    if i <2:
        continue
    print()
    print('row, ',i)
    print('=========')
    print()
    print(r.shape, x[i, 10, 0], r)

In [ ]:
list(combinations(range(5), 2))

In [ ]:
%%cython -a
cimport cython
cimport numpy as np
import numpy as np
from scipy.misc import comb
from itertools import chain
from functools import lru_cache
from scipy.sparse import csr_matrix

ctypedef int wtype
cdef char* wtype_s = 'i4'

@cython.cdivision(True)
cdef inline long comb_count_safe(long n, long k):
    cdef long i, prod = 1
    for i in range(k):
        # The bracketing is super-important. Order of operations matters.
        prod = (prod * (n - i)) / (i + 1)
        #print(n-i, i+1, prod)
    return prod
            
cdef inline object comb_cy2(wtype n):
    cdef int i, j, row = 0
    cdef wtype[:,:] out = np.zeros((comb_count_safe(n,2), 2), dtype=wtype_s)
    for i in range(n):
        for j in range(i+1,n):
            #print(i,j)
            out[row,0] = i
            out[row,1] = j
            row += 1
    return np.array(out)
          
def make_lookup_array(wtype max_n):
    cdef int i, j, k, n, row = 0
    cdef int maxcol = comb_count_safe(max_n, 2)
    cdef wtype[:,:,:] out = np.zeros((max_n+1, maxcol + 1, 2), dtype=wtype_s)
    cdef wtype[:,:] tmp
    for i in range(1, max_n + 1):
        n = comb_count_safe(i, 2)
        tmp = comb_cy2(i)
        for j in range(n):
            for k in range(2):
                out[i, j, k] = tmp[j, k]
        out[i, maxcol, 0] = n
    return np.array(out)

def method_sparse_cy2(
            int[:, :] context_blocks_array, 
            int max_words_per_block=40, 
            int max_section_length=int(1e7)):
    """
    Series of sparse matrix constructions.
    
    max_section_length is a setting.  Tweak to trade-off CPU vs RAM.
    """    
    
    cdef wtype[:,:,:] lookup = make_lookup_array(max_words_per_block)
    # Max combinations possible in each block
    cdef int p = comb(max_words_per_block, 2)     
    
    # Buffers 
    cdef np.ndarray ones = np.ones(max_section_length, dtype=wtype_s)
    cdef np.ndarray co = np.zeros((max_section_length, 2), dtype=wtype_s)
    cdef wtype[:, :] new_entries = np.zeros((p, 2), dtype=wtype_s)
    cdef wtype[:, :] indices = np.zeros((p, 2), dtype=wtype_s)
    cdef long end = 0  # Keep track of position in the allocation array 
    
    # The max number of unique words.  Might need to go up.
    # Sets num rows and cols for the output sparse matrix
    cdef long ns = 2**16  # (65536) 
    # Output. Stores co-occurrence totals between word pairs.
    # The datatype determines the max count possible, and also the 
    # memory cost of the sparse matrix.  'u2' is quite aggressively
    # small. u4 shouldn't be much worse.
    m = csr_matrix((ns, ns), dtype=wtype_s)  # 

    cdef int i, j, k, nn, num_words, n = context_blocks_array.shape[0]
    cdef int[:] block
    cdef int[:,:] pbuffer = np.zeros((p, 2), dtype=wtype_s)
    print('***')
    
    for block in context_blocks_array:
        # Combinations of words in this block.
        # Find the number of words in this block
        for num_words in range(max_words_per_block):
            if block[num_words] == -1:
                break    
        # Based on the number of words, look up the indices for the 
        # combinations.
        indices = lookup[num_words]
        # Now that we have the indices, make the array of actual
        # co-occurrences.
        for i in range(num_words):
            for j in range(2):
                new_entries[i, j] = indices[i, j]
        #print('new_entries', new_entries.shape, np.array(new_entries))
        nn = new_entries[p, 0]
        #print('nn',nn)
        #return
        # Now write the new batch of co-occurrences into the big array.
        co[end:end+nn, :] = new_entries[:nn]
        # Move the "current position" marker
        end += nn
        # Buffer might be full
        full = end > max_section_length - p  # Account for next iteration fill-up, worst case
        if full:
            m += csr_matrix((ones[:end], (co[:end, 0], co[:end, 1])), (ns, ns))
            end = 0 # Reset back to start
    
    if end > 0:
        m += csr_matrix((ones[:end], (co[:end, 0], co[:end, 1])), (ns, ns))
    return m    

In [ ]:
try:
    del m
except:
    pass

print('Length of context_blocks_array:',len(context_blocks_array))
print(context_blocks_array.shape, context_blocks_array.dtype)
with Stats():
    #m = method_sparse_cy1(context_blocks_array, max_section_length=int(1e7))
    m = method_sparse_cy2(context_blocks_array)
    
print('Total co-occurences: {:,}'.format(m.sum()))
print('Shape of sparse matrix:', m.shape)
print('Size of sparse matrix: {:,.2f} MB'.format(m.data.nbytes/1024/1024))

In [ ]:
m.nonzero()

In [ ]:
try:
    del m
except:
    pass

print('Length of context_blocks_array:',len(new_cba))
with Stats():
    m = method_sparse_cy1(new_cba, max_section_length=int(1e7))
    
print('Total co-occurences: {:,}'.format(m.sum()))
print('Size of sparse matrix: {:,.2f} MB'.format(m.data.nbytes/1024/1024))